### ML Setup

In [1]:
import sys
import typing

import picard
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

rng_seed = 90
torch.manual_seed(rng_seed)

if torch.cuda.is_available():
    device = torch.device("cuda:0")

print(device)

cuda:0


In [2]:
from pathlib import Path
import mne

eegmmidb_path = Path.cwd() / "eeg-motor-movementimagery-dataset-1.0.0" / "files"
example_edf_path = eegmmidb_path / "S001" / "S001R03.edf"

raw = mne.io.read_raw_edf(example_edf_path, preload=False)

print(raw)

# ---- 2. Print metadata ----
print("\nSampling rate:", raw.info["sfreq"])
print("Channels:", raw.info["ch_names"])
print(raw.annotations)
print("Number of channels:", raw.info["nchan"])
print("Duration (sec):", raw.n_times / raw.info["sfreq"])


events, event_id = mne.events_from_annotations(raw)
print(events)
# print(event_id)

tmin = 0
tmax = 4
epochs = mne.Epochs(
    raw,
    events,
    event_id=event_id,
    tmin=tmin,
    tmax=tmax,
    baseline=None,
    preload=True
)

print(epochs.get_data())

Extracting EDF parameters from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R03.edf...
Setting channel info structure...
Creating raw.info structure...
<RawEDF | S001R03.edf, 64 x 20000 (125.0 s), ~52 KiB, data not loaded>

Sampling rate: 160.0
Channels: ['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']
<Annotations | 30 segments: T0 (15), T1 (8), T2 (7)>
Number of channels: 64
Duration (sec): 125.0
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
[[  

In [31]:
from typing import List
import re
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader

train_subjects = [i for i in range(1, 100)] # 1-99
test_subjects = [i for i in range(100, 110)] # 100-109

mne.set_log_level("ERROR")

# def ema_normalize(x, alpha=0.001, eps=1e-6):
#     """
#     x shape: (trials, channels, time)
#     """
#     n_trials, n_channels, n_times = x.shape
#
#     mean = np.zeros((n_trials, n_channels))
#     var = np.ones((n_trials, n_channels))
#
#     out = np.zeros_like(x)
#
#     for t in range(n_times):
#         sample = x[:, :, t]
#
#         mean = (1 - alpha) * mean + alpha * sample
#         var = (1 - alpha) * var + alpha * (sample - mean) ** 2
#
#         out[:, :, t] = (sample - mean) / np.sqrt(var + eps)
#
#     return out
# def normalize_trial(x):
#     mean = x.mean(axis=1, keepdims=True)
#     std = x.std(axis=1, keepdims=True)
#     return (x - mean) / (std + 1e-6)

lrRun = {3, 4, 7, 8, 11, 12}
bothRun = {5, 6, 9, 10, 13, 14}

def extract_run_number(edf_path: str) -> int:
    name = Path(edf_path).name
    m = re.search(r"R(\d+)\.edf$", name, flags=re.IGNORECASE)
    if not m:
        raise ValueError(f"Cannot parse run number from filename: {name}")
    return int(m.group(1))

def preprocess_one_edf(edf_path, l_freq=8.0, h_freq=40.0,tmin=0.0, tmax=4.0,run_ica=True, random_state=1):

    edf_path = Path(edf_path)

    run = extract_run_number(edf_path)

    raw = mne.io.read_raw_edf(edf_path, preload=True)

    raw.pick_types(eeg=True)
    raw.filter(l_freq, h_freq)
    raw.set_eeg_reference("average", projection=False, verbose="error")

    # print(f"channel names: {raw.ch_names}")

    labels = {np.str_("T0"), np.str_("T1"), np.str_("T2")}
    # if not labels.issubset(event_id.keys()):
    #     raise RuntimeError(f"[{edf_path.name}] Missing {labels - set(event_id.keys())}. Found: {list(event_id.keys())}")
    if run in lrRun:
        semantic = {np.str_("T0"): "rest", np.str_("T1"): "left_fist", np.str_("T2"): "right_fist"}
    elif run in bothRun:
        semantic = {np.str_("T0"): "rest", np.str_("T1"): "both_fists",np.str_("T2"): "both_feet"}
    else:
        return None

    # ICA
    if run_ica:
        ica = mne.preprocessing.ICA(
            n_components=20,
            method="picard",
            random_state=random_state,
            max_iter=500,
        )

        ica.fit(raw, verbose="error")

        frontal = [ch for ch in ["Fp1", "Fp2", "AFp1", "AFp2"] if ch in raw.ch_names]

        if frontal:
            eog_proxy = raw.copy().pick_channels(frontal).get_data().mean(axis=0)
            sources = ica.get_sources(raw).get_data()

            corr = np.array([
                np.corrcoef(sources[i], eog_proxy)[0, 1]
                for i in range(sources.shape[0])
            ])

            bad_ics = np.where(np.abs(corr) > 0.3)[0]
            ica.exclude = bad_ics.tolist()
            raw = ica.apply(raw, verbose="error")

    # Events / epochs
    events, event_id = mne.events_from_annotations(raw)

    sem_event_id = {semantic[k]: event_id[k] for k in labels}

    epochs = mne.Epochs(
        raw, events, event_id=sem_event_id,
        tmin=tmin, tmax=tmax,
        baseline=None,
        preload=True,
        reject_by_annotation=True,
        verbose="error"
    )

    return epochs

class EEGMMIDBDataset(Dataset):
    def __init__(self, path: Path, subjects: List[int]):
        self.samples = []
        for subject in subjects:
            sub_folder = path / f"S{subject:03d}"

            for fname in os.listdir(sub_folder):
                if fname.endswith(".edf"):
                    X, y = self.extract_trials_from_edf(sub_folder / fname)
                    if X is None:
                        continue
                    print(f"Finished extracting from {sub_folder / fname}")
                    self.samples.extend(zip(X, y))
                    # for i in range(len(X)):
                    #     # X_norm = normalize_trial(X[i])
                    #     self.samples.append((X[i], y[i]))


        print(f"Total samples: {self.__len__()}")

    def __len__(self):
        return len(self.samples)

    def extract_trials_from_edf(self, edf_path):
        raw = mne.io.read_raw_edf(edf_path, preload=True)
        events, event_id = mne.events_from_annotations(raw)

        epochs = mne.Epochs(
            raw,
            events,
            event_id,
            tmin=0,
            tmax=4,
            baseline=None,
            preload=True,
            reject_by_annotation=True
        )

        X = epochs.get_data()

        if (X.shape[2] != 641):
            return None, None

        y = epochs.events[:, -1] - 1
        return X, y
        # epochs = preprocess_one_edf(edf_path)
        # if epochs is None:
        #     return None, None
        #
        # X = epochs.get_data()
        #
        # if (X.shape[2] != 641):
        #     return None, None
        #
        # y = epochs.events[:, -1] - 1
        #
        # return X, y

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

loader_train = DataLoader(
    EEGMMIDBDataset(eegmmidb_path, train_subjects),
    batch_size=64,
    shuffle=True,
    num_workers=0
)
loader_test = DataLoader(
    EEGMMIDBDataset(eegmmidb_path, test_subjects),
    batch_size=64,
    shuffle=True,
    num_workers=0
)


Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R01.edf
Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R02.edf
Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R03.edf
Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R04.edf
Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R05.edf
Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R06.edf
Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movementimagery-dataset-1.0.0\files\S001\S001R07.edf
Finished extracting from C:\Users\mbaza\PycharmProjects\demysml-ml1\eeg-motor-movem

In [32]:
import torch
import torch.nn as nn


class EEGNet(nn.Module):

    def __init__(self, nb_classes: int, in_channels: int):
        super().__init__()

        F1 = 8
        D = 2
        F2 = F1 * D
        fs = 160 # Sampling frequency
        kernel_length = fs // 2

        self.block1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernel_length), padding=(0, kernel_length // 2), bias=False),
            nn.BatchNorm2d(F1),

            nn.Conv2d(F1, F1 * D, (in_channels, 1), groups=F1, bias=False),
            nn.BatchNorm2d(F1 * D),

            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(0.25)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(F1 * D, F1 * D, (1, 16), padding=(0, 8), groups=F1 * D, bias=False),
            nn.Conv2d(F1 * D, F2, 1, bias=False),

            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(0.25)
        )

        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Linear(F2, nb_classes)

    def forward(self, x):
        x = x.unsqueeze(1)

        x = self.block1(x)
        x = self.block2(x)

        x = self.adaptive_pool(x)

        x = x.flatten(start_dim=1)
        return self.classifier(x)


In [33]:
# Training
dtype = torch.float32

def train_model(model, train_loader, optimizer, epochs=10):
    model = model.to(device)
    for e in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()

            x = x.to(device, dtype=dtype)
            y = y.to(device, dtype=torch.long)

            # if t == 0:
            #     print("\n--- Batch Stats ---")
            #     print("shape:", x.shape)
            #     print("mean:", x.mean().item())
            #     print("std:", x.std().item())
            #     print("min:", x.min().item())
            #     print("max:", x.max().item())
            #     print("labels:", torch.unique(y))

            scores = model(x)

            if torch.isnan(scores).any():
                print("NaNs detected in model output!")
                return

            loss = F.cross_entropy(scores, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {e+1}/{epochs}, Loss {loss.item():.4f}")

def check_accuracy(model, loader, num_classes=3):
    num_correct = 0
    num_samples = 0
    model.eval()

    class_correct = torch.zeros(num_classes, device=device)
    class_total = torch.zeros(num_classes, device=device)

    with torch.no_grad():
        for t, (x, y) in enumerate(loader):
            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)

            for i in range(num_classes):
                mask = (y == i)
                class_correct[i] += (preds[mask] == y[mask]).sum()
                class_total[i] += mask.sum()


        acc = float(num_correct) / num_samples
        print('Got %d / %d correct of val set (%.2f)' % (num_correct, num_samples, 100 * acc))
        per_class_acc = class_correct / class_total
        for i, acc in enumerate(per_class_acc):
            print(f'Class {i} Accuracy: {acc*100:.2f}%')

In [34]:
model = EEGNet(nb_classes=3, in_channels=64).to(device)

check_accuracy(model, loader_test)

optimizer = torch.optim.Adamax(model.parameters(), lr=5e-5, weight_decay=1e-7)

train_model(model, loader_train, optimizer=optimizer, epochs=10)

check_accuracy(model, loader_test)

torch.save(model.state_dict(), 'model-epoch10.pt')

Got 1627 / 3222 correct of val set (50.50)
Class 0 Accuracy: 100.00%
Class 1 Accuracy: 0.00%
Class 2 Accuracy: 0.00%
Epoch 1/10, Loss 1.0271
Epoch 2/10, Loss 1.0597
Epoch 3/10, Loss 0.9516
Epoch 4/10, Loss 1.0368
Epoch 5/10, Loss 0.9744
Epoch 6/10, Loss 1.1458
Epoch 7/10, Loss 1.0524
Epoch 8/10, Loss 1.0263
Epoch 9/10, Loss 0.9149
Epoch 10/10, Loss 0.9660
Got 1614 / 3222 correct of val set (50.09)
Class 0 Accuracy: 98.03%
Class 1 Accuracy: 1.37%
Class 2 Accuracy: 1.01%
